
# Refined Forecasting Model

Pipeline to engineer advanced features (holidays, lag, rolling windows) and tune XGBoost to minimise MAPE on daily flights.


In [1]:

import re
from pathlib import Path

import joblib
import numpy as np
import pandas as pd
from pandas.tseries.holiday import USFederalHolidayCalendar
import xgboost as xgb
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.model_selection import TimeSeriesSplit, RandomizedSearchCV
from sklearn.metrics import make_scorer


In [2]:

DATA_DIR = Path('../data')
CSV_GLOB = 'Plant*.csv'

frames = []
for path in sorted(DATA_DIR.glob(CSV_GLOB)):
    match = re.search(r"Plant\s*([0-9]+)", path.stem)
    if not match:
        continue
    plant_id = int(match.group(1))
    df = pd.read_csv(path, encoding='utf-8-sig', low_memory=False)
    df['plant_id'] = plant_id
    frames.append(df)

if not frames:
    raise RuntimeError('No se encontraron archivos Plant*.csv en ../data.')

df_raw = pd.concat(frames, ignore_index=True)
df_raw.columns = [col.strip().lower().replace(' ', '_') for col in df_raw.columns]
df_raw['day'] = pd.to_datetime(df_raw['day'], errors='coerce')
df_raw = df_raw.dropna(subset=['day']).sort_values(['plant_id', 'day']).reset_index(drop=True)

print('Registros por planta:')
print(df_raw.groupby('plant_id')['day'].agg(['min', 'max', 'count']))
df_raw.head()


Registros por planta:
                min        max  count
plant_id                             
1        2023-01-02 2025-04-30    845
2        2023-01-02 2025-08-31    973


,day,flights,passengers,max_capacity,plant_id
0,2023-01-02,352,94434.0,99351.0,1
1,2023-01-03,356,92710.0,99616.0,1
2,2023-01-04,358,88888.0,99437.0,1
3,2023-01-05,365,88276.0,99174.0,1
4,2023-01-06,367,89086.0,100611.0,1


In [3]:

df = df_raw.copy()

# Temporal context
df['dayofweek'] = df['day'].dt.dayofweek
df['month'] = df['day'].dt.month
df['quarter'] = df['day'].dt.quarter
df['year'] = df['day'].dt.year
df['dayofyear'] = df['day'].dt.dayofyear
df['weekofyear'] = df['day'].dt.isocalendar().week.astype(int)
df['is_weekend'] = (df['dayofweek'] >= 5).astype(int)

# Holidays
cal = USFederalHolidayCalendar()
holidays = cal.holidays(start=df['day'].min(), end=df['day'].max())
df['is_holiday'] = df['day'].isin(holidays).astype(int)

# Cyclical encoding
df['sin_dayofyear'] = np.sin(2 * np.pi * df['dayofyear'] / 365.0)
df['cos_dayofyear'] = np.cos(2 * np.pi * df['dayofyear'] / 365.0)

LAGS = [1, 2, 3, 7, 14]
for lag in LAGS:
    df[f'lag_{lag}'] = df.groupby('plant_id')['flights'].shift(lag)

ROLL_WINDOWS = [3, 7, 14, 28]
for window in ROLL_WINDOWS:
    shifted = df.groupby('plant_id')['flights'].shift(1)
    df[f'roll_mean_{window}'] = shifted.rolling(window=window, min_periods=2).mean()
    df[f'roll_std_{window}'] = shifted.rolling(window=window, min_periods=2).std()

for window in ROLL_WINDOWS:
    df[f'roll_std_{window}'] = df[f'roll_std_{window}'].fillna(0)

required = [f'lag_{lag}' for lag in LAGS] + [f'roll_mean_{window}' for window in ROLL_WINDOWS]
df_features = df.dropna(subset=required).reset_index(drop=True)

FEATURES = [
    'plant_id', 'dayofweek', 'month', 'quarter', 'year', 'dayofyear', 'weekofyear',
    'is_weekend', 'is_holiday', 'sin_dayofyear', 'cos_dayofyear'
] + [f'lag_{lag}' for lag in LAGS] + [f'roll_mean_{window}' for window in ROLL_WINDOWS] + [f'roll_std_{window}' for window in ROLL_WINDOWS]

TARGET = 'flights'
print('Total registros tras ingeniería de variables:', len(df_features))
df_features.head()


Total registros tras ingeniería de variables: 1790


,day,flights,passengers,max_capacity,plant_id,dayofweek,month,quarter,year,dayofyear,...,lag_7,lag_14,roll_mean_3,roll_std_3,roll_mean_7,roll_std_7,roll_mean_14,roll_std_14,roll_mean_28,roll_std_28
0,2023-01-16,375,88694.0,102596.0,1,0,1,1,2023,16,...,383.0,352.0,376.666667,10.066446,374.000000,13.916417,368.000000,13.190906,368.000000,13.190906
1,2023-01-17,380,81292.0,103739.0,1,1,1,1,2023,17,...,382.0,356.0,375.666667,10.016653,372.857143,13.371968,369.642857,12.456738,368.466667,12.838929
2,2023-01-18,380,79001.0,102618.0,1,2,1,1,2023,18,...,346.0,358.0,380.333333,5.507571,372.571429,13.163803,371.357143,12.080545,369.187500,12.734304
3,2023-01-19,370,83558.0,101797.0,1,3,1,1,2023,19,...,377.0,365.0,378.333333,2.886751,377.428571,6.106203,372.928571,11.631947,369.823529,12.605729
4,2023-01-20,381,86145.0,104273.0,1,4,1,1,2023,20,...,378.0,367.0,376.666667,5.773503,376.428571,6.729466,373.285714,11.445043,369.833333,12.229424


In [4]:

last_date = df_features['day'].max()
cutoff_date = last_date - pd.DateOffset(months=3)

train_df = df_features[df_features['day'] <= cutoff_date].copy()
test_df = df_features[df_features['day'] > cutoff_date].copy()

print(f"Train: {train_df['day'].min()} -> {train_df['day'].max()} | {len(train_df)} registros")
print(f"Test : {test_df['day'].min()} -> {test_df['day'].max()} | {len(test_df)} registros")

X_train = train_df[FEATURES]
y_train = train_df[TARGET]
X_test = test_df[FEATURES]
y_test = test_df[TARGET]


Train: 2023-01-16 00:00:00 -> 2025-05-31 00:00:00 | 1698 registros
Test : 2025-06-01 00:00:00 -> 2025-08-31 00:00:00 | 92 registros


In [5]:

param_distributions = {
    'n_estimators': [300, 400, 500, 600],
    'max_depth': [3, 4, 5, 6],
    'learning_rate': [0.03, 0.05, 0.07],
    'subsample': [0.8, 0.9, 1.0],
    'colsample_bytree': [0.8, 0.9, 1.0],
    'min_child_weight': [1, 3, 5],
    'gamma': [0, 0.1, 0.3],
    'reg_lambda': [1, 2, 3]
}

base_model = xgb.XGBRegressor(
    objective='reg:squarederror',
    tree_method='hist',
    random_state=42,
    n_jobs=1
)

ts_cv = TimeSeriesSplit(n_splits=5)
scorer = make_scorer(mean_absolute_percentage_error, greater_is_better=False)

random_search = RandomizedSearchCV(
    estimator=base_model,
    param_distributions=param_distributions,
    n_iter=20,
    scoring=scorer,
    cv=ts_cv,
    verbose=1,
    random_state=42,
    n_jobs=1
)

random_search.fit(X_train, y_train)

best_model = random_search.best_estimator_
print('Mejores hiperparámetros encontrados:')
print(random_search.best_params_)


Fitting 5 folds for each of 20 candidates, totalling 100 fits


Mejores hiperparámetros encontrados:
{'subsample': 1.0, 'reg_lambda': 1, 'n_estimators': 600, 'min_child_weight': 3, 'max_depth': 6, 'learning_rate': 0.03, 'gamma': 0.1, 'colsample_bytree': 0.8}


In [6]:

y_pred_test = best_model.predict(X_test)
y_pred_train = best_model.predict(X_train)

test_mape = mean_absolute_percentage_error(y_test, y_pred_test)
train_mape = mean_absolute_percentage_error(y_train, y_pred_train)

print(f"Test MAPE: {test_mape * 100:.2f}%")
print(f"Train MAPE: {train_mape * 100:.2f}%")

if test_mape <= 0.02:
    print("\n✅ Objetivo de MAPE ≤ 2% alcanzado.")
else:
    print("\n⚠️  Objetivo de MAPE ≤ 2% no alcanzado con el dataset actual.")


Test MAPE: 9.76%
Train MAPE: 13647242644684800.00%

⚠️  Objetivo de MAPE ≤ 2% no alcanzado con el dataset actual.


In [7]:

# Entrenar con todo el conjunto y guardar artefacto
best_model.fit(df_features[FEATURES], df_features[TARGET])
output_path = Path('../backend/models/refined_flights_forecasting_model.pkl')
output_path.parent.mkdir(parents=True, exist_ok=True)

payload = {
    'model': best_model,
    'features': FEATURES,
    'best_params': random_search.best_params_,
    'train_mape': train_mape,
    'test_mape': test_mape,
    'cutoff_date': cutoff_date
}

joblib.dump(payload, output_path)
print(f"Modelo guardado en {output_path.resolve()}")


Modelo guardado en C:\Users\Lenovo\Desktop\UdeCodes\backend\models\refined_flights_forecasting_model.pkl



## Resultados y Observaciones

- Se incorporaron variables de calendario, festivos federales, rezagos y ventanas móviles.
- Se utilizó `RandomizedSearchCV` con validación `TimeSeriesSplit` (5 particiones) para ajustar hiperparámetros de XGBoost.
- Con los datos disponibles, el MAPE en el conjunto de prueba continúa por encima del objetivo de 2% (≈10%).
- Recomendación: depurar/normalizar los datos históricos (por ejemplo, coherencia de Plant 2) o explorar modelos específicos por planta que reduzcan la variabilidad.
